# Imports

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import string, random

# make

In [2]:
!cd .. && make clean && make all
!cd instancias && rm *.txt

g++ -std=c++11 -Wall -c src/HashMapConcurrente.cpp -o build/HashMapConcurrente.o -lpthread
g++ -std=c++11 -Wall -c src/CargarArchivos.cpp -o build/CargarArchivos.o -lpthread
g++ -std=c++11 -Wall -o build/ContarPalabras src/ContarPalabras.cpp build/HashMapConcurrente.o build/CargarArchivos.o -lpthread
rm build/HashMapConcurrente.o build/CargarArchivos.o


## Definitions

In [3]:
BIN = "./../build/ContarPalabras"
TEST_FILE = "../data/corpus"
runs = []

In [5]:
def correr_experimento(t_leer, t_max, files):
    # Leer archivo de la instancia.
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen([BIN] + [t_leer, t_max] + files, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    time_l, time_max, _ = tuple(process.stderr.read().split("\n"))
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [float(time_l), float(time_max)]

## txt generator: a agus le gusta que le digan

In [7]:
def txt(txtname, size):
    file = open(f'instancias/{txtname}.txt', 'w+')
    for _ in range(size):
        word = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(3, 9)))
        file.write(word + '\n')
    

for x in range(20 + 1):
    txt(x, 5000)

In [8]:
correr_experimento('1', '1', ["0.txt"])

TypeError: expected str, bytes or os.PathLike object, not int

## Todos los tipos

In [6]:
cantArchivos = 8
archivos = []
for _ in range(cantArchivos):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')

for threadsLectura in range(1, cantArchivos + 1):
    for threadsMax in range(1, 26 + 1):
        runs.append([str(threadsLectura), str(threadsMax), archivos, '1'])

# experimento : threadsMaximo

In [7]:
cantArchivos = 8
archivos = []
for _ in range(cantArchivos):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')

for threadsMax in range(1, 27):
    runs.append(['1', str(threadsMax), archivos, '2'])

# experimento : threadsLectura

In [8]:
cantArchivos = 20
archivos = []
for _ in range(cantArchivos):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')

for threadsLectura in range(1, cantArchivos + 1):
    runs.append([str(threadsLectura), '1', archivos, '3'])

# variando la cantidad de archivos por cantidad de threads:

In [9]:
archivos = []
for cantArchivos in range(1 , 13 + 1):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')
    for threadsLectura in range(1, cantArchivos + 1):
        runs.append([str(threadsLectura), '1', archivos.copy(), '4'])

In [10]:
run = runs[0]
correr_experimento(run[0], run[1], run[2])

[1005.27, 2.5244]

### Ejectuar los experimentos:

In [11]:
# [cantThreads][cantFiles][cantThreadsMax][TIEMPO_MAX][TIEMPO_LEER]
columnas = ["threadsLectura", "threadsMaximo", "cantArchivos", "TIEMPO_MAX", "TIEMPO_CARGAR", "EXPERIMENTO"];
filas = [];
numero = 1
T = 50 #Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for run in runs:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(runs)))
    numero += 1
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiemposMax = []
    tiemposCargar = []
    tiempo = []
    #print(experimento[4], experimento[5])
    for _ in range(0, T):
        tiempo = correr_experimento(run[0], run[1], run[2])
        tiemposCargar.append(tiempo[0])
        tiemposMax.append(tiempo[1])
    tiempoCargar = np.median(tiemposCargar);
    tiempoMax = np.median(tiemposMax);
    filas.append([run[0], run[1], len(run[2]), tiempoMax, tiempoCargar, run[3]]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 345/345'